## File names etc.

In [1]:
# angle_rep 0: Full unit circle, angle_rep 1: Right half plane, angle_rep 2: Regular angle
angle_rep = 0
#img_list_file = 'list_of_img_in_val_set_18-03.csv'
img_list_file = '../list_of_img_in_OP_val_set_14-04.csv'

path_img_folder = '../../../03 Data/Dataset2_onPallet/'

model_folder = "./02 On Pallet Dataset/01 Snapshots/"
model_name = "OP_Full_SmoothL1_05_resnet50_csv_25.h5"

## Imports

In [2]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from numpy import genfromtxt
import time
import json
import copy

# use this to change which GPU to use
gpu = 0
# set the modified tf session as backend in keras  
#setup_gpu(gpu)  #NOTICE: enable when using paperspace server!!

Using TensorFlow backend.


## Annotation Loading Functions

In [3]:
def convert_bp_to_aabb(points):
    max_x = max(points[:,0])
    min_x = min(points[:,0])
    max_y = max(points[:,1])
    min_y = min(points[:,1])

    return (min_x, min_y, max_x, max_y)

def read_annotations_from_json(img_name):
    with open(path_img_folder + img_name.strip('.png') + '.json') as json_file:
        data = json.load(json_file)
        shapes_list = data['shapes']
        bbox_list = []
        angle_list = []
        for annotation in shapes_list: 
            #rect = cv2.boundingRect(np.float32(annotation['points']))
            rot_rect = cv2.minAreaRect(np.float32(annotation['points']))
            if rot_rect[-2][0] < rot_rect[-2][1]:
                rot_rect = (rot_rect[0],(rot_rect[-2][1],rot_rect[-2][0]),rot_rect[-1]+90)
            # make sure that no angle is above 90 or below -90
            if rot_rect[-1] > 90:
                rot_rect = (rot_rect[0],rot_rect[1],rot_rect[-1]-180)
            if rot_rect[-1] < -90:
                rot_rect = (rot_rect[0],rot_rect[1],rot_rect[-1]+180)
            #angle_list.append(rot_rect[-1]/180*np.pi)
            bp = cv2.boxPoints(rot_rect)
            angle_list.append(rot_rect[-1])
            bbox_list.append(convert_bp_to_aabb(bp))
    return bbox_list, angle_list; 

## Rectangle Functions

In [4]:
def area(BB):
    width = abs(BB[0] - BB[2])
    height = abs(BB[1] - BB[3])
    return height * width

def intersection(BB1, BB2):
    # find coordiantes of intersection rectangle 
    (x1, y1, x2, y2) = BB1
    (x3, y3, x4, y4) = BB2
    if x1 > x4 or x3 > x2 or y3 > y2 or y1 > y4:
        return 0
    x5 = max(x1, x3);
    y5 = max(y1, y3);
    x6 = min(x2, x4);
    y6 = min(y2, y4);  
    BB = (x5, y5, x6, y6)
    # TODO: check if no intersection exists
    return area(BB)

def union(BB1, BB2): 
    return area(BB1) + area(BB2) - intersection(BB1, BB2)

def IoU(BB1, BB2):
    return intersection(BB1, BB2)/union(BB1, BB2)

## Prediction Manipulation Functions

In [5]:
def filter_predictions(bboxes, scores, labels, angles, score_threshold):
    filtered_bboxes = []
    filtered_scores = []
    filtered_labels = []
    filtered_angles = []
    
    for bbox, score, label, angle in zip(bboxes, scores, labels, angles):
        if score < score_threshold: 
            break
            
        filtered_bboxes.append(bbox)
        filtered_scores.append(score)
        filtered_labels.append(label)
        filtered_angles.append(angle)

    return filtered_bboxes, filtered_scores, filtered_labels, filtered_angles

def format_angles(angles, rep):
    formatted_angles = []
    if rep == 0:
        for angle in angles:
            formatted_angles.append(np.arctan2(angle[1],angle[0])/2/np.pi*180)
    elif rep == 1:
        for angle in angles:
            formatted_angles.append(np.arctan2(angle[1],angle[0])/np.pi*180)
    elif rep == 2:
        formatted_angles.append(angle/np.pi*180)
    return formatted_angles 

## Evaluation Functions

In [6]:
# creates a matrix of IoU between prediction and annotation rectangles. The predictions and annotations with the highest IoU
# are matched. The matching indices and associated IoU-scores are reported as (pred_idx, anno_idx, IoU)
def match_annotations(predictions, annotations):
    #create the matrix
    IoU_mat = np.zeros((len(predictions), len(annotations)))
    # fill matrix with IoU values
    for pred_idx, pred in enumerate(predictions):
        for anno_idx, anno in enumerate(annotations):
            IoU_mat[pred_idx, anno_idx] = IoU(pred, anno)
    
    # create output list
    annotation_matches = []
    if IoU_mat.size != 0:
        max_idx = np.unravel_index(np.argmax(IoU_mat, axis=None), IoU_mat.shape)
        max_IoU = IoU_mat[max_idx]
        while max_IoU > 0:
            # set the chosen rows and columns to 0
            IoU_mat[max_idx[0],:] = 0
            IoU_mat[:,max_idx[1]] = 0
            # append the indices and IoU to result
            annotation_matches.append((max_idx[0],max_idx[1],max_IoU))
            max_idx = np.unravel_index(np.argmax(IoU_mat, axis=None), IoU_mat.shape)
            max_IoU = IoU_mat[max_idx]
    
    return annotation_matches

# Returns (T. Pos, F. Pos, F. Neg) for certain threshold
def evaluate_bb(predictions, annotations, annotation_matches, threshold):
    i = 0
    true_pos = 0
    while (i < len(annotation_matches)) and (annotation_matches[i][2]>threshold):
        i+=1
        true_pos+=1
    false_pos = len(predictions) - true_pos;
    false_neg = len(annotations) - true_pos;
    return true_pos, false_pos, false_neg

# Returns the sum of angle errors and squared sum of angle errors for true positives
# Errors are assumed in degrees
def evaluate_angle(predictions, annotations, annotation_matches, threshold):
    i = 0
    angle_err = 0
    angle_err_sqr = 0
    while (i < len(annotation_matches)) and (annotation_matches[i][2]>threshold):
        pred_idx, anno_idx = annotation_matches[i][0:-1]
        diff = annotations[anno_idx] - predictions[pred_idx]
        diff = min(abs(diff), abs(diff-180), abs(diff+180)) # makes sure that the sigularity does not skew the results. Assumes degrees
        angle_err += diff
        angle_err_sqr += diff**2
        i+=1
    return angle_err, angle_err_sqr

def evaluate_center_point(predictions, annotations, annotation_matches, threshold):
    i = 0
    center_pt_err = 0
    center_pt_err_sqr = 0
    while (i < len(annotation_matches)) and (annotation_matches[i][2]>threshold):
        pred_idx, anno_idx = annotation_matches[i][0:-1]
        pred_center_x = (predictions[pred_idx][0]+predictions[pred_idx][2])/2
        pred_center_y = (predictions[pred_idx][1]+predictions[pred_idx][3])/2
        anno_center_x = (annotations[anno_idx][0]+annotations[anno_idx][2])/2
        anno_center_y = (annotations[anno_idx][1]+annotations[anno_idx][3])/2     
        diff = np.sqrt((pred_center_x - anno_center_x)**2 + (pred_center_y - anno_center_y)**2) 
        center_pt_err += diff
        center_pt_err_sqr += diff**2
        i+=1
    return center_pt_err, center_pt_err_sqr

# finds annotation matches and calls evaluate_bb and evaluate_angle for several IoU-thresholds
# ongoing_results is a tuple (total_T_pos, total_F_pos, total_F_neg, angle_err_sum, angle_err_sqr_sum)
def evaluate_range(ongoing_results, box_preds, angle_preds, box_annos, angle_annos, thresholds):
    annotation_matches = match_annotations(box_preds, box_annos)
    for idx, thresh in enumerate(thresholds):
        true_pos, false_pos, false_neg = evaluate_bb(box_preds, box_annos, annotation_matches, thresh)
        angle_err, angle_err_sqr = evaluate_angle(angle_preds, angle_annos, annotation_matches, thresh)
        center_pt_err, center_pt_err_sqr = evaluate_center_point(box_preds, box_annos, annotation_matches, thresh)
        ongoing_results[idx][0] += true_pos 
        ongoing_results[idx][1] += false_pos
        ongoing_results[idx][2] += false_neg
        ongoing_results[idx][3] += angle_err
        ongoing_results[idx][4] += angle_err_sqr
        ongoing_results[idx][5] += center_pt_err
        ongoing_results[idx][6] += center_pt_err_sqr
        
    return ongoing_results

#Returns tuple (thresh, prec, rec, f1, avg_ang_err, std_ang_err, avg_center_err, std_center_err, t_pos, f_pos, f_neg)
def get_result(final_results, thresholds):
    result = []
    avg_prec = 0
    avg_rec = 0
    avg_avg_ang_err = 0
    avg_std_ang_err = 0
    avg_avg_center_err = 0
    avg_std_center_err = 0
    for final_res, thresh in zip(final_results, thresholds):
        precision = final_res[0]/(final_res[0]+final_res[1])
        recall = final_res[0]/(final_res[0]+final_res[2])
        f1 = 2*precision*recall/(precision+recall)
        avg_ang_err = final_res[3]/final_res[0]
        std_ang_err = np.sqrt((final_res[4]/final_res[0])-(avg_ang_err**2))
        avg_center_err = final_res[5]/final_res[0]
        std_center_err = np.sqrt((final_res[6]/final_res[0])-(avg_center_err**2))
        result.append((thresh,precision, recall, f1, avg_ang_err, std_ang_err, avg_center_err, std_center_err, final_res[0], final_res[1], final_res[2]))
        avg_prec += precision
        avg_rec += recall
        avg_avg_ang_err += avg_ang_err
        avg_std_ang_err += std_ang_err
        avg_avg_center_err += avg_center_err
        avg_std_center_err += std_center_err
    # append the average at the end 
    l = len(thresholds)
    result.append(('Avg', avg_prec/l, avg_rec/l, 2*avg_prec/l*avg_rec/l/(avg_prec/l+avg_rec/l), avg_avg_ang_err/l, avg_std_ang_err/l,avg_avg_center_err/l, avg_std_center_err/l, '--','--','--'))
    return result
    

## File Generation

In [7]:
def write_results(folder, name, results, time):
    with open(folder + name, 'w') as f:
        f.write('IoU, Prec., Rec., F1, ang. err., std. ang. err., cent. err., std. cent. err. ,F. Neg, F. Pos, T. Pos \n')
        for result in results:
            f.write(str(result[0])+', '+str(result[1])+', '+str(result[2])+', '+str(result[3])+', '+str(result[4])+', '+str(result[5])+ ', '+str(result[6])+', '+str(result[7])+', '+str(result[10])+', '+str(result[9])+', '+str(result[8])+'\n')
        f.write('Time: '+str(time))

## Model Setup

In [8]:
# import model
model_path = os.path.join(model_folder, model_name);
model = models.load_model(model_path, backbone_name='resnet50');

# If model is not converted to inference model, use line below: 
model = models.convert_model(model);

# Mapping of model output and classes
labels_to_names = {0: 'Brick'};

tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 , -14.25438 ,  28.50876 ,  14.25438 ],
       [-35.918785, -17.959393,  35.918785,  17.959393],
       [-16.      , -16.      ,  16.      ,  16.      ],
       [-20.158737, -20.158737,  20.158737,  20.158737],
       [-25.398417, -25.398417,  25.398417,  25.398417],
       [-11.313708, -22.627417,  11.313708,  22.627417],
       [-14.25438 , -28.50876 ,  14.25438 ,  28.50876 ],
       [-17.959393, -35.918785,  17.959393,  35.918785]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-45.254833, -22.627417,  45.254833,  22.627417],
       [-57.01752 , -28.50876 ,  57.01752 ,  28.50876 ],
       [-71.83757 , -35.918785,  71.83757 ,  35.918785],
       [-32.      , -32.      ,  32.      ,  32.      ],
       [-40.317474, -40.317474,  40.317474,  40.317474],
       [-50.796833, -50.796833,  50.7

## Evaluation

In [9]:
# import data
file = open(img_list_file)
file_paths = list(file)
#file_paths = file_paths[0:30]

IoU_thresholds = np.arange(0.5,1,0.05)

ongoing_results = np.zeros((len(IoU_thresholds),7))

total_time = 0
for path in file_paths:
    image = read_image_bgr(path_img_folder + path.strip('\n'))

    start = time.perf_counter()
    # preprocess image 
    # TODOD: check if preprocess_image convert the image to RGB format
    image = preprocess_image(image)
    image, scale = resize_image(image)
    # process image 
    boxes, scores, labels, angles = model.predict_on_batch(np.expand_dims(image, axis=0))
    # correct for image scale
    boxes /= scale
    boxes, scores, labels, angles = filter_predictions(boxes[0], scores[0], labels[0], angles[0], 0.9)
    angles = format_angles(angles, angle_rep)
    end = time.perf_counter()
    total_time += end-start
    
    ##load annotations
    box_annos, angle_annos = read_annotations_from_json(path.strip('\n'))
    ongoing_results =  evaluate_range(ongoing_results, boxes, angles, box_annos, angle_annos, IoU_thresholds)
final_results = get_result(ongoing_results, IoU_thresholds)
write_results('./02 On Pallet Dataset/', model_name[0:-3]+'results_center.csv', final_results, total_time)    